In [ ]:
%pip install patool

In [ ]:
!unrar x -kb -or -- /content/You.rar /content/
!unrar x -kb -or -- '/content/Not You.rar'  /content/

In [ ]:
import os

directory = '/content/content/You'
if not os.path.exists(directory):
    print(f"Directory '{directory}' does not exist.")
else:
    num_files = sum(len(files) for _, _, files in os.walk(directory))
    print(f"Number of files in '{directory}': {num_files}")

In [ ]:
import os

directory = '/content/content/not You'
if not os.path.exists(directory):
    print(f"Directory '{directory}' does not exist.")
else:
    num_files = sum(len(files) for _, _, files in os.walk(directory))
    print(f"Number of files in '{directory}': {num_files}")

In [ ]:
!mkdir '/content/train'

!mkdir '/content/train/You'
!mkdir '/content/train/Not You'

!mkdir '/content/test'

!mkdir '/content/test/You'
!mkdir '/content/test/Not You'

In [ ]:
# first shuffle the files in '/content/content/You' and '/content/content/not You' and then
# copy 70% of files in '/content/content/You' in '/content/train/You',
# 70% of files in '/content/content/not You' in '/content/train/Not You',
# 30% of files in '/content/content/You' in '/content/test/You',
# 30% of files in '/content/content/not You' in '/content/test/Not You'

import random

you_dir = '/content/content/You'
not_you_dir = '/content/content/not You'

train_you_dir = '/content/train/You'
train_not_you_dir = '/content/train/Not You'
test_you_dir = '/content/test/You'
test_not_you_dir = '/content/test/Not You'

# Get list of files and shuffle them
you_files = [os.path.join(you_dir, f) for f in os.listdir(you_dir) if os.path.isfile(os.path.join(you_dir, f))]
not_you_files = [os.path.join(not_you_dir, f) for f in os.listdir(not_you_dir) if os.path.isfile(os.path.join(not_you_dir, f))]

random.shuffle(you_files)
random.shuffle(not_you_files)

# Calculate split points
you_train_split = int(0.7 * len(you_files))
not_you_train_split = int(0.7 * len(not_you_files))

# Split files into train and test sets
you_train_files = you_files[:you_train_split]
you_test_files = you_files[you_train_split:]

not_you_train_files = not_you_files[:not_you_train_split]
not_you_test_files = not_ishan_files[not_you_train_split:]

# Copy files to train and test directories
for f in you_train_files:
    shutil.copy(f, train_you_dir)

for f in you_test_files:
    shutil.copy(f, test_you_dir)

for f in not_you_train_files:
    shutil.copy(f, train_not_you_dir)

for f in not_you_test_files:
    shutil.copy(f, test_not_you_dir)

# Print number of files copied
print(f"Copied {len(you_train_files)} files to {train_you_dir}")
print(f"Copied {len(you_test_files)} files to {test_you_dir}")
print(f"Copied {len(not_you_train_files)} files to {train_not_you_dir}")
print(f"Copied {len(not_you_test_files)} files to {test_not_you_dir}")

In [ ]:
import os

directory = '/content/train/You'
if not os.path.exists(directory):
    print(f"Directory '{directory}' does not exist.")
else:
    num_files = sum(len(files) for _, _, files in os.walk(directory))
    print(f"Number of files in '{directory}': {num_files}")

directory2 = '/content/train/Not You'
if not os.path.exists(directory2):
    print(f"Directory '{directory2}' does not exist.")
else:
    num_files = sum(len(files) for _, _, files in os.walk(directory2))
    print(f"Number of files in '{directory2}': {num_files}")

directory3 = '/content/test/You'
if not os.path.exists(directory3):
    print(f"Directory '{directory3}' does not exist.")
else:
    num_files = sum(len(files) for _, _, files in os.walk(directory3))
    print(f"Number of files in '{directory3}': {num_files}")

directory4 = '/content/test/Not You'
if not os.path.exists(directory4):
    print(f"Directory '{directory4}' does not exist.")
else:
    num_files = sum(len(files) for _, _, files in os.walk(directory4))
    print(f"Number of files in '{directory4}': {num_files}")

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img

In [ ]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255,
    fill_mode='nearest'
)

In [ ]:
batch_size = 16

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#Generating train and test data

train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size = (224, 224),
    batch_size = batch_size,
    class_mode = 'binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/test',
    target_size = (224, 224),
    batch_size = batch_size,
    class_mode = 'binary'
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
#Making the CNN model

model = Sequential()

model.add(Conv2D(3, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(6, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Training for 10 epochs
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

In [ ]:
#test one image and evaluate if the model is predicting correctly

import numpy as np
from PIL import Image

# Choose a random image file from the test set for testing
test_image_path = '/content/test/You/daylight_face_0.jpg' # Or test_not_ishan_files[0]

# Load and preprocess the image
img = load_img(test_image_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.  # Rescale

# Make prediction
prediction = model.predict(img_array)

# Get the predicted class label
predicted_class_index = int(round(prediction[0][0]))

# Get the class labels from the generator (assuming binary classification)
class_labels = list(train_generator.class_indices.keys())
predicted_class_label = class_labels[predicted_class_index]

# Determine the actual class label based on the file path
actual_class_label = 'You' if 'Not' not in test_image_path else 'Not You'

# Print the results
print(f"Testing image: {test_image_path}")
print(f"Predicted class: {predicted_class_label}")
print(f"Actual class: {actual_class_label}")

# Evaluate if the prediction is correct
if predicted_class_label == actual_class_label:
    print("Prediction is correct!")
else:
    print("Prediction is incorrect.")



In [ ]:
# determine the accuracy of the model on the test set

loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Accuracy on the test set: {accuracy}")


In [ ]:
model.save('MyModel.keras')